In [46]:
import glob
from keras.models import Sequential
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras import optimizers

In [47]:
from chat_wordvec import * 

In [48]:
maxsent = 50
num_features = 300

### Bentuk menjadi encoded form

In [49]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from numpy import argmax

def list_to_onehot(lista):
    global integer_encoded, onehot_encoded, end_dense_layer
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(lista)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    end_dense_layer = len(onehot_encoded)
    return onehot_encoded

def lst_to_onehot_encoded(lista,listb):
    result = []
    for i,la in enumerate(lista):
        cont = listb.index(la)
        print(i,la,cont,'->',onehot_encoded[cont], integer_encoded[cont])
        result.append(onehot_encoded[cont])
    return result
        
#inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
#print(onehot_encoded)

#print(onehot_encoded[aa.index("saya masih muda")])

In [50]:
def permutate_list(lista,listb):
    "Permutate 2 lists to two outputs with subs"
    tmp_x = []
    tmp_y = []
    for la in lista:
        for lb in listb:
            tmp_x.append(la)
            tmp_y.append(lb)
    return tmp_x, tmp_y

def unwrap_list(lista):
    "Unwrap sub of a list, to make a flat list"
    t_x = []
    for tx in lista:
        for t in tx:
            t_x.append(t)
    return t_x

def make_uniq_list(lista):
    return list(set(lista))

## PROCESS ALL TEXT FILES

In [51]:
fls = glob.glob('./dialogs/*.yml')

trn_x = []
trn_y = []

tmp_x = []
tmp_y = []


for fl in fls:
    with open(fl,'r') as f:
        txt = f.readlines()


    for i,tx in enumerate(txt):
        if tx[0] == "A":
            tmp_x.append(tx[3:-1])
        elif tx[0] == "B":
            tmp_y.append(tx[3:-1])
        else:
            tmp_x, tmp_y = permutate_list(tmp_x,tmp_y)
            trn_x.append(tmp_x)
            trn_y.append(tmp_y)
            tmp_x = []
            tmp_y = []

    trn_x = unwrap_list(trn_x)
    trn_y = unwrap_list(trn_y)

### Membuat encoding respons robot

In [52]:
respons = make_uniq_list(trn_y)
one_hot = list_to_onehot(respons)
train_y = lst_to_onehot_encoded(trn_y,respons)
train_y = np.array(train_y)

import pickle
pickle.dump(respons, open("respons.p","wb"))
pickle.dump(integer_encoded, open("integer_encoded.p","wb"))
pickle.dump(onehot_encoded, open("onehot_encoded.p",'wb'))

0 ya 23 -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1.] [26]
1 ya 23 -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1.] [26]
2 ya 23 -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1.] [26]
3 apa 14 -> [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.] [0]
4 apa 14 -> [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.] [0]
5 apa 14 -> [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.] [0]
6 kenapa 13 -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.] [9]
7 nama saya cipi 12 -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.] [10]
8 nama saya cipi 12 -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.] [10]
9 nama saya cipi 12 -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0

### rubah semua menjadi matrix

In [53]:
for i,o in enumerate(respons):
    print(i, o, '->', one_hot[i])

0 di mesin komputer -> [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
1 saya melayani majikan saya -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0.]
2 kabar baik-baik saja. -> [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
3 saya di panggil cipi, panggil saja -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
4 saya terdiri dari syaraf otak digital dan kode pithon -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0.]
5 saya disini-sini saja -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
6 saya tinggal disini saja -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0.]
7 he -> [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
8 terima kasih, saya juga senang -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [54]:
print(respons)
print(integer_encoded)
print(onehot_encoded)

['di mesin komputer', 'saya melayani majikan saya', 'kabar baik-baik saja.', 'saya di panggil cipi, panggil saja', 'saya terdiri dari syaraf otak digital dan kode pithon', 'saya disini-sini saja', 'saya tinggal disini saja', 'he', 'terima kasih, saya juga senang', 'saya suka kamu', 'bumi itu bulat', 'saya masih muda', 'nama saya cipi', 'kenapa', 'apa', 'cuaca menurut saya sangat menarik, hangat dan semilir', 'saya suka ngobrol', 'saya senang berkenalan dengan anda', 'kasihan, apa yang terjadi', 'sampai jumpa lagi', 'saya lagi santai saja', 'asik banget nih', 'senang berkenalan', 'ya', 'hehehe', 'saya bisa bahasa indonesia, inggris dan komputer', 'saya juga tertarik']
[[ 4]
 [18]
 [ 7]
 [13]
 [22]
 [14]
 [23]
 [ 5]
 [25]
 [20]
 [ 2]
 [17]
 [10]
 [ 9]
 [ 0]
 [ 3]
 [21]
 [19]
 [ 8]
 [11]
 [16]
 [ 1]
 [24]
 [26]
 [ 6]
 [12]
 [15]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 

## cekidot data

In [55]:
for i,t in enumerate(trn_x):
    print(i, t, '->' , trn_y[i], train_y[i])

0 CV CV -> ya [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1.]
1 CV CV CV -> ya [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1.]
2 CV -> ya [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1.]
3 tv tv tv -> apa [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
4 tv tv -> apa [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
5 tv -> apa [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
6 Chibi -> kenapa [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
7 siapa kamu -> nama saya cipi [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
8 apa panggilan kamu -> nama saya cipi [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
9 nama -> nama saya cipi [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0

In [56]:
train_x = []
for tx in trn_x:
    train_x.append(swv_ar(twl(tx,rempunct=True,flat=True), maxword=maxsent, vecsize=num_features,frontpad=True))
train_x = np.array(train_x)

Problem at phrase: chibi
Problem at phrase: hehehe


In [57]:
print(train_x.shape)
print(train_y.shape)

(46, 50, 300)
(46, 27)


In [58]:
model=Sequential()
model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="sigmoid", units=300, dropout=0.2))
model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=False, activation="sigmoid", units=300,dropout=0.2))
model.add(Dense(300, activation="relu"))
model.add(Dense(end_dense_layer,activation="sigmoid"))
#model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
#model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#cosine_proximity
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

In [59]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 50, 300)           721200    
_________________________________________________________________
lstm_6 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_5 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_6 (Dense)              (None, 27)                8127      
Total params: 1,540,827
Trainable params: 1,540,827
Non-trainable params: 0
_________________________________________________________________


In [60]:
model.fit(train_x,train_y, epochs=300)
model.save("./model/chitchat-300-1.mdl")

Epoch 1/300
46/46 [==============================] - 1s 31ms/step - loss: -0.1915 - acc: 0.0870
Epoch 2/300
46/46 [==============================] - 0s 8ms/step - loss: -0.2169 - acc: 0.1304
Epoch 3/300
46/46 [==============================] - 0s 8ms/step - loss: -0.2303 - acc: 0.1304
Epoch 4/300
46/46 [==============================] - 0s 8ms/step - loss: -0.2351 - acc: 0.1304
Epoch 5/300
46/46 [==============================] - 0s 8ms/step - loss: -0.2374 - acc: 0.1304
Epoch 6/300
46/46 [==============================] - 0s 8ms/step - loss: -0.2382 - acc: 0.1304
Epoch 7/300
46/46 [==============================] - 0s 8ms/step - loss: -0.2382 - acc: 0.1304
Epoch 8/300
46/46 [==============================] - 0s 8ms/step - loss: -0.2385 - acc: 0.1304
Epoch 9/300
46/46 [==============================] - 0s 8ms/step - loss: -0.2389 - acc: 0.1304
Epoch 10/300
46/46 [==============================] - 0s 9ms/step - loss: -0.2392 - acc: 0.1304
Epoch 11/300
46/46 [============================

46/46 [==============================] - 0s 8ms/step - loss: -0.8908 - acc: 0.8913
Epoch 87/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9104 - acc: 0.8913
Epoch 88/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9104 - acc: 0.9348
Epoch 89/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9151 - acc: 0.9565
Epoch 90/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9226 - acc: 0.9565
Epoch 91/300
46/46 [==============================] - 0s 7ms/step - loss: -0.9206 - acc: 0.9130
Epoch 92/300
46/46 [==============================] - 0s 7ms/step - loss: -0.9138 - acc: 0.8913
Epoch 93/300
46/46 [==============================] - 0s 7ms/step - loss: -0.9274 - acc: 0.9348
Epoch 94/300
46/46 [==============================] - 0s 7ms/step - loss: -0.9331 - acc: 0.9565
Epoch 95/300
46/46 [==============================] - 0s 7ms/step - loss: -0.9344 - acc: 0.9348
Epoch 96/300
46/46 [==============================] -

46/46 [==============================] - 0s 8ms/step - loss: -0.9856 - acc: 0.9783
Epoch 171/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9859 - acc: 0.9783
Epoch 172/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9843 - acc: 0.9565
Epoch 173/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9857 - acc: 0.9565
Epoch 174/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9856 - acc: 0.9565
Epoch 175/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9857 - acc: 0.9783
Epoch 176/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9846 - acc: 0.9783
Epoch 177/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9857 - acc: 0.9783
Epoch 178/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9845 - acc: 0.9783
Epoch 179/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9857 - acc: 0.9783
Epoch 180/300
46/46 [=======================

46/46 [==============================] - 0s 8ms/step - loss: -0.9866 - acc: 0.9783
Epoch 255/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9867 - acc: 1.0000
Epoch 256/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9865 - acc: 0.9783
Epoch 257/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9866 - acc: 0.9783
Epoch 258/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9869 - acc: 0.9783
Epoch 259/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9864 - acc: 0.9783
Epoch 260/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9866 - acc: 0.9783
Epoch 261/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9868 - acc: 0.9783
Epoch 262/300
46/46 [==============================] - 0s 7ms/step - loss: -0.9867 - acc: 0.9783
Epoch 263/300
46/46 [==============================] - 0s 8ms/step - loss: -0.9865 - acc: 0.9783
Epoch 264/300
46/46 [=======================

In [31]:
def pred(sentence):
    """memprediksi masuk intent apakah sebuah kalimat."""
    predictions=model.predict(np.array([swv_ar(twl(sentence,rempunct=True,flat=True),maxword=maxsent, vecsize=num_features)]))
    cls_pred = np.argmax(predictions,axis=1)
    #print(predictions)
    
    #ini sangat tricky -> dari prediksi balikkan ke integer_encodednya dulu baru 
    #cari ke indexnya dari response
    
    jawaban = respons[list(integer_encoded).index(cls_pred[0])]

    result={'jawaban': jawaban , 'score': predictions.max(), 'sentence': sentence, 'cls:': cls_pred}
    return print(result)

In [32]:
for i in trn_x:
    pred(i)

{'jawaban': 'ya', 'score': 0.41950944, 'sentence': 'CV CV', 'cls:': array([24], dtype=int64)}
{'jawaban': 'ya', 'score': 0.39679566, 'sentence': 'CV CV CV', 'cls:': array([24], dtype=int64)}
{'jawaban': 'ya', 'score': 0.33523494, 'sentence': 'CV', 'cls:': array([24], dtype=int64)}
Problem at phrase: chibi
{'jawaban': 'hehehe', 'score': 0.97283936, 'sentence': 'Chibi', 'cls:': array([5], dtype=int64)}
{'jawaban': 'nama saya cipi', 'score': 0.026028536, 'sentence': 'siapa kamu', 'cls:': array([8], dtype=int64)}
{'jawaban': 'nama saya cipi', 'score': 0.01866609, 'sentence': 'apa panggilan kamu', 'cls:': array([8], dtype=int64)}
{'jawaban': 'nama saya cipi', 'score': 0.026476283, 'sentence': 'nama', 'cls:': array([8], dtype=int64)}
{'jawaban': 'nama saya cipi', 'score': 0.02587177, 'sentence': 'namanya siapa', 'cls:': array([8], dtype=int64)}
{'jawaban': 'nama saya cipi', 'score': 0.028158775, 'sentence': 'siapa nama kamu', 'cls:': array([8], dtype=int64)}
{'jawaban': 'nama saya cipi', 'sc

In [45]:
%%time
pred('sedang apa')

{'jawaban': 'saya lagi santai saja', 'score': 0.018149177, 'sentence': 'sedang apa', 'cls:': array([14], dtype=int64)}
Wall time: 15 ms
